In [2]:
# Configure the necessary Spark environment
import os
import sys

# Spark home
spark_home = os.environ.get("SPARK_HOME")

# If Spark V1.4.x is detected, then add ' pyspark-shell' to
# the end of the 'PYSPARK_SUBMIT_ARGS' environment variable
spark_release_file = spark_home + "/RELEASE"
if os.path.exists(spark_release_file) and "Spark 1.4" in open(spark_release_file).read():
    pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
    if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
    os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

# Add the spark python sub-directory to the path
sys.path.insert(0, spark_home + "/python")

# Add the py4j to the path.
# You may need to change the version number to match your install
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.8.2.1-src.zip"))

# Initialize PySpark to predefine the SparkContext variable 'sc'
execfile(os.path.join(spark_home, "python/pyspark/shell.py"))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.4.1
      /_/

Using Python version 2.7.9 (default, Dec 18 2014 16:57:52)
SparkContext available as sc, SQLContext available as sqlContext.


In [15]:
print sys.path.insert(0, "C:\Anaconda\lib\site-packages\numpy")
import numpy

None


In [3]:
sc

In [16]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.util import MLUtils
import pandas as pd


In [5]:
# Load data file
data_tree = sc.textFile('c:\Users\AD\Documents\BCfiles\KaggleEEG\df_for_tree_small.csv')

#pd.read_csv
#sc.textFile

In [6]:
#Removing header of the csv.
data_tree = data_tree.filter(lambda l: "class_all" not in l)


In [7]:
data_tree.take(5)

[u'0,-10.83805,-9.63178,-10.99165,-10.99361',
 u'0,-9.74717,-10.22009,-10.90319,-10.60587',
 u'0,-10.88845,-10.79459,-11.20667,-11.77245',
 u'0,-9.59486,-9.85923,-10.31219,-11.23681',
 u'0,-8.62924,-10.02107,-11.18154,-10.34676']

In [8]:
# split -> convert to num 
def parsePoint(line):
    line = line.split(',')
    line = [float(x) for x in line]
    features = line[1:]
    target = line[0]
    return LabeledPoint(target,features)
    

In [9]:
line = u'0,-11.56247,-11.0003,-11.58689,-11.65356'
line = line.split(',')
line_test = [float(x) for x in line]
print line_test
print line_test[0]
print line_test[1:]

[0.0, -11.56247, -11.0003, -11.58689, -11.65356]
0.0
[-11.56247, -11.0003, -11.58689, -11.65356]


In [9]:
modeldata = data_tree.map(parsePoint)
modeldata.first()

LabeledPoint(0.0, [-10.83805,-9.63178,-10.99165,-10.99361])

In [92]:
import numpy

In [31]:
(training, test) = modeldata.randomSplit([0.6,0.4])
#trainingData, testData = modeldata.randomSplit([0.8, 0.2])

In [37]:
training.take(5)

[LabeledPoint(0.0, [-10.83805,-9.63178,-10.99165,-10.99361]),
 LabeledPoint(0.0, [-9.74717,-10.22009,-10.90319,-10.60587]),
 LabeledPoint(0.0, [-9.59486,-9.85923,-10.31219,-11.23681]),
 LabeledPoint(0.0, [-8.62924,-10.02107,-11.18154,-10.34676]),
 LabeledPoint(0.0, [-11.88613,-11.41672,-11.70803,-11.15817])]

In [50]:
#testing changing to 7 classes 
t2= training.filter(lambda x: x.label<7)

In [51]:
model = DecisionTree.trainClassifier(t2, numClasses=7, categoricalFeaturesInfo={},impurity='gini', maxDepth=5, maxBins=32)


In [54]:
# Evaluate model on test instances and compute test error
predictions = model.predict(test.map(lambda x: x.features))
labelsAndPredictions = test.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(test.count())
print('Test Error = ' + str(testErr))
print('Learned classification tree model:')
print(model.toDebugString())

Test Error = 0.0968068930563
Learned classification tree model:
DecisionTreeModel classifier of depth 5 with 59 nodes
  If (feature 0 <= -10.26922)
   If (feature 2 <= -10.89916)
    If (feature 3 <= -10.61495)
     If (feature 2 <= -11.63137)
      If (feature 2 <= -12.57792)
       Predict: 0.0
      Else (feature 2 > -12.57792)
       Predict: 0.0
     Else (feature 2 > -11.63137)
      If (feature 2 <= -11.42089)
       Predict: 0.0
      Else (feature 2 > -11.42089)
       Predict: 0.0
    Else (feature 3 > -10.61495)
     If (feature 3 <= -9.9352)
      If (feature 0 <= -10.7456)
       Predict: 0.0
      Else (feature 0 > -10.7456)
       Predict: 0.0
     Else (feature 3 > -9.9352)
      If (feature 3 <= -9.63767)
       Predict: 0.0
      Else (feature 3 > -9.63767)
       Predict: 0.0
   Else (feature 2 > -10.89916)
    If (feature 3 <= -12.06371)
     If (feature 2 <= -10.79069)
      If (feature 1 <= -11.6688)
       Predict: 4.0
      Else (feature 1 > -11.6688)
       Pre